In [113]:
### Library Import Initialization

import numpy as np
import math
from math import *
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy.stats import multivariate_normal
from scipy.integrate import dblquad
from math import exp, sqrt, pi
import warnings
warnings.filterwarnings("ignore")

In [114]:
### Function to Import Stock Tickers and Calculate Final Stock Price
def import_stock_data(tickers, start_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date)['Adj Close']
    return data

tickers = 'GOOG'
stock_data = import_stock_data(tickers, '2018-01-01')
# Get the Current Stock Price (Starting Node of Tree)
S_0 = stock_data[tickers].iloc[-1]
S_0


[*********************100%%**********************]  1 of 1 completed


151.94000244140625

In [115]:
### Sigma Calculation 
def compute_sigma(data):
    # Compute the standard deviation of returns
    sigma = np.std(data) / 100
    return sigma

get_sigma = compute_sigma(stock_data)
sigma = get_sigma.values[0]
sigma

0.32973889585923516

In [116]:
### Cumulative standard normal distribution
def cdf(x):
    return (1.0 + erf(x / np.sqrt(2.0))) / 2.0

### Compute phi Function
def phi(S_0, T, y, H, I, r, b, sigma):
    '''
    ϕ(S,T,y,H,I) represents the price of an American option with a continuous barrier. It incorporates the effects of early exercise 
    and the presence of a barrier, which can be either up-and-out, up-and-in, down-and-out, or down-and-in, depending on the 
    relative positions of the barrier level and the current asset price
    The function takes the following parameters:
    S_0: Current price of the underlying asset
    T: Time to expiration of the option
    y: The "nu" parameter, or gamma, which is often related to the cost of carry or dividend yield
    H: The barrier level
    I: The rebate, which is a cash payment given if the barrier is hit
    '''
    lam = (-r + y * b + 0.5 * y * (y - 1) * sigma**2) * T

    d_1 = (np.log(S_0 / H) + (b + (y - 0.5) * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d_2 = d_1 - sigma * np.sqrt(T)

    k = (2 * b / (sigma**2)) + (2 * y - 1)

    phi = np.exp(lam) * (S_0**y) * (cdf(-d_1) - (((I/S_0)**k) * cdf(-d_2)))
    
    return phi


In [117]:
### Cunulative Bivariate Normal Distribution Function
# Code from https://stackoverflow.com/questions/41596143/python-bivariate-normal-cdf-with-variable-upper-bound
def genz_2004_cbnd(dh, dk, r):
    # dh and dk get signs flipped right away
    dh = float(-dh)
    dk = float(-dk)

    if (dh == np.inf) or (dk == np.inf): 
        return 0
    
    else:
        if dh == - np.inf:
            if dk == - np.inf:
                return 1
            else:
                return norm.cdf(- dk)
        if dk == - np.inf:
            return norm.cdf(- dh)
        else:
            if r == 0:
                return norm.cdf(- dh)*norm.cdf(- dk)
            else:
                    tp=2*np.pi
                    h=dh
                    k=dk
                    hk=h*k
                    bvn=0
                    if abs(r) < 0.3:
                        w=np.array([0.1713244923791705,0.3607615730481384,0.4679139345726904])
                        x=np.array([0.9324695142031522,0.6612093864662647,0.238619186083197])
                    else:
                        if abs(r) < 0.75:
                            w=np.array([0.04717533638651177,0.1069393259953183,0.1600783285433464,0.2031674267230659,0.2334925365383547,0.2491470458134029])
                            x=np.array([0.9815606342467191,0.904117256370475,0.769902674194305,0.5873179542866171,0.3678314989981802,0.1252334085114692])
                        else:
                            w=np.array([0.01761400713915212,0.04060142980038694,0.06267204833410905,0.08327674157670475,0.1019301198172404,0.1181945319615184,0.1316886384491766,0.1420961093183821,0.1491729864726037,0.1527533871307259])
                            x=np.array([0.9931285991850949, 0.9639719272779138, 0.9122344282513259,0.8391169718222188, 0.7463319064601508, 0.6360536807265150,0.5108670019508271,0.3737060887154196,0.2277858511416451,0.07652652113349733])
                    w = np.concatenate((w, w), axis=0)
                    x = np.concatenate((1 - x, 1 + x), axis=0)

                    if abs(r) < 0.925:
                        hs=(h*h+k*k) / 2
                        asr=np.arcsin(r) / 2
                        sn=np.sin(asr*x)
                        bvn=np.dot(np.exp((sn*hk-hs)/(1-sn**2)),w.T)
                        bvn=bvn*asr/tp + norm.cdf(-h)*norm.cdf(-k)

                    else:
                        if r < 0:
                            k=- k
                            hk=- hk
                            
                        if abs(r) < 1:
                            as1=1 - r ** 2
                            a=np.sqrt(as1)
                            bs=(h - k) ** 2
                            asr=- (bs / as1 + hk) / 2
                            c=(4 - hk) / 8
                            d=(12 - hk) / 80

                            if asr > - 100:
                                bvn= a*np.exp(asr)*(1-c*(bs-as1)*(1-d*bs)/3+c*d*as1**2)
                            if hk > - 100:
                                b=np.sqrt(bs)
                                sp=np.sqrt(tp)*norm.cdf(-b/a)
                                bvn=bvn - np.exp(-hk/2)*sp*b*( 1 - c*bs*(1-d*bs)/3 )
                            a=a / 2
                            xs=(a*x) ** 2
                            asr=- (bs / xs + hk) / 2
                            ix=asr > - 100
                            xs=xs[ix]
                            sp=( 1 + c*xs*(1+5*d*xs) )
                            rs=np.sqrt(1 - xs)
                            ep=np.exp(- (hk / 2)*xs / (1 + rs) ** 2) / rs
                            bvn=( a*( np.dot((np.exp(asr[ix])*(sp-ep)),w[ix].T) ) - bvn )/tp

                        if r > 0:
                            bvn=bvn + norm.cdf(- max(h,k))

                        else:
                            if h >= k:
                                bvn=- bvn

                            else:
                                if h < 0:
                                    L=norm.cdf(k) - norm.cdf(h)
                                else:
                                    L=norm.cdf(- h) - norm.cdf(- k)
                                bvn=L - bvn

                    return max(0,min(1,bvn))

In [118]:
### Compute ksi Function
def ksi(S_0, T, y, H, I_2, I_1, t_1, r, b, sigma):
    '''
    psi is a crucial part of the BS call option approximation formula as it incorporates the barrier feature and early exercise behavior into the option pricing calculation

    S_0: Current price of the underlying asset (stock price).
    T: Time to expiration of the option (in years).
    y: The "gamma" parameter, which may represent the cost of carry or dividend yield.
    H: The barrier level. This is the price level at which the barrier option comes into effect.
    I_2: Second flat boundary. This is related to the barrier structure and may have specific conditions for early exercise or option value adjustments.
    I_1​: First flat boundary. Similar to I_2​, this is also part of the barrier structure and affects the option's behavior.
    t_1​: Time boundary. This represents the time at which the second flat boundary (I_2​) comes into effect.
    r: Risk-free interest rate. This is typically the interest rate on a risk-free asset, such as a government bond.
    b: Cost of carry. This can include costs such as storage costs, financing costs, or dividend yield, depending on the context.
    sigma: Volatility of the underlying asset's returns. This measures the variability of the asset's price over time.
    '''
    e_1 = (np.log(S_0 / I_1) + (b + (y-1/2)*sigma**2) * t_1) / (sigma * np.sqrt(t_1))
    e_2 = (np.log(I_2**2 / (S_0*I_1)) + (b + (y-1/2)*sigma**2) * t_1) / (sigma * np.sqrt(t_1))
    e_3 = (np.log(S_0 / I_1) - (b + (y-1/2)*sigma**2) * t_1) / (sigma * np.sqrt(t_1))
    e_4 = (np.log(I_2**2 / (S_0*I_1)) - (b + (y-1/2)*sigma**2) * t_1) / (sigma * np.sqrt(t_1))

    f_1 = (np.log(S_0 / H) + (b + (y-1/2)*sigma**2) * T) / (sigma * np.sqrt(T))
    f_2 = (np.log(I_2**2 / (S_0*H)) + (b + (y-1/2)*sigma**2) * T) / (sigma * np.sqrt(T))
    f_3 = (np.log(S_0 / S_0*H) - (b + (y-1/2)*sigma**2) * T) / (sigma * np.sqrt(T))
    f_4 = (np.log((S_0*I_1)**2 / (H*I_2)**2) - (b + (y-1/2)*sigma**2) * T) / (sigma * np.sqrt(T))      

    k = (2*b / sigma**2) + (2*y + 1)

    ksi = np.exp((b - r) * T) * S_0**y * (
        genz_2004_cbnd(-e_1, -f_1, -np.sqrt(t_1 / T)) + genz_2004_cbnd(-e_2, -f_2, -np.sqrt(t_1 / T)) - I_2**(k) * genz_2004_cbnd(-e_3, -f_3, -np.sqrt(t_1 / T)) - I_1**(k) * genz_2004_cbnd(-e_4, -f_4, -np.sqrt(t_1 / T))
    )
    #test1 = np.exp((b - r) * T) * S_0**y
    #print('test1:', test1)
    #test2 = genz_2004_cbnd(-e_1, -f_1, -np.sqrt(t_1 / T))
    #print('test2:', test2)
    #test3 = genz_2004_cbnd(-e_2, -f_2, -np.sqrt(t_1 / T)) - I_2**(k)
    #print('test3:', test3)
    #test4 = genz_2004_cbnd(-e_3, -f_3, -np.sqrt(t_1 / T)) - I_1**(k)
    #print('test4:', test4)
    #print('-e_3', e_3)
    #print('-f_3', f_3)
    #print('-np.sqrt(t_1 / T))', -np.sqrt(t_1 / T))
    #print('I_1**(k)', I_1**(k))
    #test5 = genz_2004_cbnd(-e_4, -f_4, -np.sqrt(t_1 / T))
    #print('test5:', test5)
    return ksi

# Test the functions together
S_0 = 100  # Current stock price
T = 1  # Time to expiration
y = 0  # Cost of carry or dividend yield
H = 90  # Barrier level
I_2 = 110  # Second flat boundary
I_1 = 95  # First flat boundary
t_1 = 0.5  # Time boundary
r = 0.05  # Risk-free interest rate
b = 0.02  # Cost of carry
sigma = 0.2  # Volatility

result = ksi(S_0, T, y, H, I_2, I_1, t_1, r, b, sigma)
print(result)

-27.759926284422562


In [119]:
### Compute Bjerksund-Stensland Call
def bjerksund_stensland_call(S_0, K, T, r, b, sigma):
    '''
    S_0: Current price of the underlying asset (stock price).
    K: Asset strike price
    T: Time to expiration of the option (in years).
    r: Risk-free interest rate. This is typically the interest rate on a risk-free asset, such as a government bond.
    b: Cost of carry. This can include costs such as storage costs, financing costs, or dividend yield, depending on the context.
    sigma: Volatility of the underlying asset's returns. This measures the variability of the asset's price over time.
    '''
    # Calculate Beta, the exponent parameter in the Black-Scholes formula
    Beta = (0.5 - (b / sigma ** 2)) + np.sqrt(((b / sigma ** 2) - 0.5) ** 2 + 2 * r / sigma ** 2)

    # Calculate B_0 and B_infty, parameters related to the option's strike price and boundaries
    B_0 = Beta / (Beta - 1) * K
    B_infty = max(K, r / (r - b) * K)

    # Calculate t_1, a time parameter used in barrier option calculations
    t_1 = 0.5 * (np.sqrt(5) - 1) * T

    # Calculate h_1 and h_2, parameters related to the option's barrier and pricing
    h_1 = -((b * t_1) + 2 * sigma * np.sqrt(t_1)) * (K ** 2 / ((B_infty - B_0) * B_0))
    h_2 = -((b * T) + 2 * sigma * np.sqrt(T)) * (K ** 2 / ((B_infty - B_0) * B_0))

    # Calculate I_1 and I_2, trigger prices used in barrier option pricing
    I_1 = B_0 + (B_infty - B_0) * (1 - np.exp(h_1))
    I_2 = B_0 + (B_infty - B_0) * (1 - np.exp(h_2))

    # Calculate alpha_1 and alpha_2, coefficients used in option pricing formulas
    alpha_1 = (I_1 - K) * (I_1) ** (-Beta)
    alpha_2 = (I_2 - K) * (I_2) ** (-Beta)
    
    # Logic to determine whether to exercise option
    if S_0 >= I_2:
        # If stock price >= upper trigger price (boundary has been met), option price = intrinsic value
        call_approx = S_0 - K
    else:
        # Compute the call option approximation using the BS formula and related functions
        call_approx = (
            alpha_2 * (S_0 ** Beta) - alpha_2 * phi(S_0, t_1, Beta, I_2, I_2, r, b, sigma) +
            phi(S_0, t_1, 1, I_2, I_2, r, b, sigma) - phi(S_0, t_1, 1, I_1, I_2, r, b, sigma) -
            K * phi(S_0, t_1, 0, I_2, I_2, r, b, sigma) + K * phi(S_0, t_1, 0, I_1, I_2, r, b, sigma) +
            alpha_1 * phi(S_0, t_1, Beta, I_1, I_2, r, b, sigma) - alpha_1 * ksi(S_0, T, Beta, I_1, I_2, I_1, t_1, r, b, sigma) +
            ksi(S_0, T, 1, I_1, I_2, I_1, t_1, r, b, sigma) - ksi(S_0, T, 1, K, I_2, I_1, t_1, r, b, sigma) -
            K * ksi(S_0, T, 0, I_1, I_2, I_1, t_1, r, b, sigma) + ksi(S_0, T, 0, K, I_2, I_1, t_1, r, b, sigma)
            )

    return t_1, h_1, h_2, I_1, I_2, call_approx


In [120]:
### Function Call
# Compute the Bjerksund-Stensland call approximation
t_1, h_1, h_2, I_1, I_2, call_approx = bjerksund_stensland_call(S_0=S_0, K=155, T=1, r=0.05, b=0, sigma=sigma)

# Print the results
print("First flat boundary (I_1):", round(I_1, 5))
print("Second flat boundary (I_2):", round(I_2, 5))
print("Time boundary (t_1):", str(round(t_1*365, 0)) + ' days')
print("h_1:", round(h_1, 5))
print("h_2:", round(h_2, 5))
print("Bjerksund-Stensland call approximation:", round(call_approx, 5))


First flat boundary (I_1): 317.70544
Second flat boundary (I_2): 326.59138
Time boundary (t_1): 226.0 days
h_1: 0.19548
h_2: 0.24865
Bjerksund-Stensland call approximation: -135.73493


In [121]:
### Test Case
# Compute the Bjerksund-Stensland call approximation
t_1, h_1, h_2, I_1, I_2, call_approx = bjerksund_stensland_call(S_0=90, K=100, T=0.1, r=0.1, b=0, sigma=0.15)

# Print the results
print("First flat boundary (I_1):", round(I_1, 5))
print("Second flat boundary (I_2):", round(I_2, 5))
print("Time boundary (t_1):", str(round(t_1*365, 0)) + ' days')
print("h_1:", round(h_1, 5))
print("h_2:", round(h_2, 5))
print("Bjerksund-Stensland call approximation: " + str(round(call_approx, 5)) + " and the correct option price should be: 0.0205")

First flat boundary (I_1): 145.35219
Second flat boundary (I_2): 147.04553
Time boundary (t_1): 23.0 days
h_1: 0.13476
h_2: 0.17142
Bjerksund-Stensland call approximation: -97.37193 and the correct option price should be: 0.0205
